# Sun2FLOPs Colab Driver

This notebook installs the package, fetches weather data, runs a single simulation, and performs a parameter sweep.

## Quick start
1. Mount your Google Drive if the repo is stored there.
2. Set `REPO_DIR` to the local folder that contains `pyproject.toml` (not a URL).
3. Run the install cell (it upgrades pvlib and installs the package).

Set your NSRDB key/email in the next cell before fetching weather.

If you're using Colab's GitHub integration, you may need to `git clone` the repo into `/content` and set `REPO_DIR` to that local folder (it should not be a URL).


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import pathlib

REPO_DIR = "/content/drive/MyDrive/sun2flops"  # TODO: update if your repo lives elsewhere
repo_path = pathlib.Path(REPO_DIR)
if not (repo_path / "pyproject.toml").exists():
    raise FileNotFoundError(
        f"pyproject.toml not found in {REPO_DIR}. Update REPO_DIR to your repo root."
    )

%cd {REPO_DIR}

In [ ]:
!pip install -U pvlib
!pip install -e .

In [ ]:
import os

os.environ["NSRDB_API_KEY"] = "YOUR_API_KEY_HERE"  # TODO: replace
os.environ["NSRDB_EMAIL"] = "YOUR_EMAIL_HERE"  # TODO: replace


In [ ]:
from sun2flops.config.models import (
    SiteConfig, WeatherConfig, PVConfig, BatteryConfig, GPUConfig,
    GovernorConfig, SimConfig, SweepConfig, FullConfig,
)
from sun2flops.data.nsrdb import load_weather_range
from sun2flops.sim.engine import run_simulation
from sun2flops.sweep.runner import run_sweep
from sun2flops.viz.plots import plot_timeseries
from sun2flops.viz.heatmaps import plot_heatmap
import json
import pandas as pd

In [ ]:
cfg = FullConfig(
    site=SiteConfig(
        name="Texas",
        latitude=31.9686,
        longitude=-99.9018,
        timezone="America/Chicago",
    ),
    weather=WeatherConfig(source="nsrdb", years=[2021]),
    pv=PVConfig(surface_tilt_deg=20, surface_azimuth_deg=180, dc_nameplate_kw=3),
    battery=BatteryConfig(energy_capacity_kwh=10, soc_init=0.5),
    gpu=GPUConfig(),
    governor=GovernorConfig(),
    sim=SimConfig(),
    sweep=SweepConfig(),
)
cfg

In [ ]:
weather = load_weather_range(cfg.site, cfg.weather)
weather.head()

In [ ]:
results, metrics = run_simulation(weather, cfg, single_year=2021)
metrics

In [ ]:
plot_timeseries(results, year=2021)

In [ ]:
sweep_df = run_sweep(weather, cfg)
sweep_df.head()

In [ ]:
plot_heatmap(sweep_df, x="pv_kw", y="batt_kwh", z="utilization")

In [ ]:
results.to_csv("simulation_timeseries.csv")
with open("simulation_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)
sweep_df.to_csv("sweep_results.csv", index=False)